In [1]:
pip install -q bitsandbytes trl 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 28.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# %%writefile train_dpo.py
from datasets import load_dataset

# Load dataset
ds = load_dataset("Jennny/helpsteer2-helpfulness-preference", split="train")

# Transform function
def transform(example):
    prompt = example["chosen"][0]["content"]
    chosen = example["chosen"][-1]["content"]
    rejected = example["rejected"][-1]["content"]

    return {
        "prompt": prompt,
        "chosen": chosen,
        "rejected": rejected,
        "margin": example.get("margin", 0)
    }

# Convert
dataset = ds.map(transform, remove_columns=ds.column_names)


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/807k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7221 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/373 [00:00<?, ? examples/s]

Map:   0%|          | 0/7221 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from trl import KTOConfig, KTOTrainer

model_name = "Qwen/Qwen3-0.6B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="bfloat16",   # or torch.bfloat16
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],  # or ["q_proj","k_proj","v_proj","o_proj"]
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)

2025-11-21 14:26:39.775583: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763735199.964593      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763735200.030612      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [4]:
from trl import DPOConfig, DPOTrainer
training_args = DPOConfig(output_dir="Qwen2-0.6B-DPO",report_to="none",    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    # max_length=512,
    # max_prompt_length=384,
    # key flag:
    precompute_ref_log_probs=True,
    bf16=True,
    gradient_checkpointing=True,)
trainer = DPOTrainer(model=model, args=training_args, processing_class=tokenizer, train_dataset=dataset)
trainer.train()

Extracting prompt in train dataset:   0%|          | 0/7221 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/7221 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/7221 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Train dataset reference log probs:   0%|          | 0/3611 [00:00<?, ?it/s]

Step,Training Loss
10,0.689400
20,0.696400
30,0.695000
40,0.690100
50,0.692500
60,0.694500
70,0.695700
80,0.696400
90,0.690800
100,0.689600


TrainOutput(global_step=1356, training_loss=0.677575833502069, metrics={'train_runtime': 30224.168, 'train_samples_per_second': 0.717, 'train_steps_per_second': 0.045, 'total_flos': 0.0, 'train_loss': 0.677575833502069, 'epoch': 3.0})

In [5]:
trainer.save_model("Qwen3-0.6B-DPO")
tokenizer.save_pretrained("Qwen3-0.6B-DPO")


('Qwen3-0.6B-DPO/tokenizer_config.json',
 'Qwen3-0.6B-DPO/special_tokens_map.json',
 'Qwen3-0.6B-DPO/chat_template.jinja',
 'Qwen3-0.6B-DPO/vocab.json',
 'Qwen3-0.6B-DPO/merges.txt',
 'Qwen3-0.6B-DPO/added_tokens.json',
 'Qwen3-0.6B-DPO/tokenizer.json')

# Evals

In [3]:
# !pip install -q lm-eval datasets accelerate transformers sentencepiece

In [4]:
KTO="AIPlans/Qwen3-0.6B-DPO"
!lm-eval \
  --model hf \
  --model_args pretrained="$KTO" \
  --tasks truthfulqa_mc2,hellaswag,arc_easy,arc_challenge,winogrande \
  --device cuda \
  --batch_size 4 \
  --output_path results_kto.json

2025-11-22 04:03:54.380231: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763784234.402646     143 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763784234.410207     143 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
INFO:lm_eval.__main__:Selected Tasks: ['arc_challenge', 'arc_easy', 'hellaswa